In [7]:
%load_ext autoreload
%autoreload 2

import h5py
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.optimize import curve_fit

mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
import util as yu

path='dat/NST_b_cD96.h5'
with h5py.File(path) as f:
    t=(f['diags/N/data/N2_N2'][:,:,0]+f['diags/N/data/N2_N2'][:,:,1])/2
    C2pt_dat=np.real(t)
    
    tfs=[int(ele.split('_')[-1]) for ele in f['diags/NJN/data'].keys()]
    tfs=list(set(tfs))
    tfs.sort()
    
    C3pt_dat_dic={tf:np.real(f['diags/NJN/data/gS_j+_deltat_{}'.format(tf)][:]) for tf in tfs}
    print(C3pt_dat_dic[12].shape)
    
    C2pt_dat_dic={tf:np.real(f['diags/N_correlated_NJN/data/dt{}_twop'.format(tf)][:]) for tf in tfs}
    print(C2pt_dat_dic[12].shape)
    
    tfs_disc=[int(ele.split('_')[-1]) for ele in f['diags/N-j/data'].keys()]
    tfs_disc=list(set(tfs_disc))
    tfs_disc.sort()
    C3pt_disc_dat_dic={tf:np.real((f['diags/N-j/data/N2_j+_N2_deltat_{}'.format(tf)][:,:,0,0]+f['diags/N-j/data/N2_j+_N2_deltat_{}'.format(tf)][:,:,1,0])/2) for tf in tfs_disc}

    vev_dat=np.real(f['VEV/j/data/j+'][:,0])
    
aMUL=0.00054
ZP=0.493
aLat=0.05692 # lattice spacing a in fm
aInvLat=1/aLat*197.3/1000 # a^{-1} in GeV


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(494, 13)
(494,)


In [8]:
C2pt_jk=yu.jackknife(C2pt_dat)
C3pt_jk_dic={tf:yu.jackknife(C3pt_dat_dic[tf]) for tf in tfs}
C2pt_jk_dic={tf:yu.jackknife(C2pt_dat_dic[tf]) for tf in tfs}
C3pt_disc_jk_dic={tf:yu.jackknife(C3pt_disc_dat_dic[tf]) for tf in tfs_disc}
vev_jk=yu.jackknife(vev_dat)

In [9]:
(Nrow,Ncol)=(1,2); (Lrow,Lcol)=(4,8)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False, gridspec_kw={'width_ratios': [2, 1]}, sharex='col', sharey=True)

tau_plat=8
chi2Size=12
xUnit=1; yUnit=aMUL*aInvLat*1000
for irow,case in enumerate(['ratio']):
    for tf in tfs:
        icol=0
        tcList=np.arange(tf+1)
        ratio_jk=[C3pt[tcList]/C2pt[tf] for C2pt,C3pt in zip(C2pt_jk,C3pt_jk_dic[tf])] \
            if case=='ratio' else [C3pt[tcList]/C2pt for C2pt,C3pt in zip(C2pt_jk_dic[tf],C3pt_jk_dic[tf])]
        ratio_jk=np.array(ratio_jk)
        ratio_mean,ratio_err=yu.jackme(ratio_jk)
        plt_x=(tcList[1:tf]-tf//2)*xUnit; plt_y=ratio_mean[1:tf]*yUnit; plt_err=ratio_err[1:tf]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
        
        icol=1
        if tf<tau_plat*2+2:
            plt_x=tf*xUnit; plt_y=ratio_mean[tf//2]*yUnit; plt_err=ratio_err[tf//2]*yUnit
            axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
        else:
            tcList=np.arange(tau_plat,tf-tau_plat+1)
            def fitfunc(pars):
                return np.zeros(len(tcList))+pars[0]
            def estimator(pars):
                return [pars[0]]
            y_jk=ratio_jk[:,tcList]; p0=[ratio_jk[0,tf//2]]
            obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,p0,estimator=estimator)
            (obs_mean,obs_err)=yu.jackme(obs_jk)
            plt_x=tf*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
            axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
            ylim=axs[irow,icol].get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
            axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),size=chi2Size,ha='center')
            
plt.tight_layout()
plt.savefig('fig/3pt_plateau.pdf')
plt.close()

In [10]:
(Nrow,Ncol)=(1,2); (Lrow,Lcol)=(4,8)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False, gridspec_kw={'width_ratios': [2, 1]}, sharex='col', sharey=True)

tau_plats=np.arange(1,13+1)
chi2Size=12
xUnit=1; yUnit=aMUL*aInvLat*1000
for irow,case in enumerate(['ratio']):
    ratio_jk_dic={}
    for tf in tfs:
        icol=0
        tcList=np.arange(tf+1)
        ratio_jk=[C3pt[tcList]/C2pt[tf] for C2pt,C3pt in zip(C2pt_jk,C3pt_jk_dic[tf])] \
            if case=='ratio' else [C3pt[tcList]/C2pt for C2pt,C3pt in zip(C2pt_jk_dic[tf],C3pt_jk_dic[tf])]
        ratio_jk=np.array(ratio_jk)
        ratio_jk_dic[tf]=ratio_jk
        ratio_mean,ratio_err=yu.jackme(ratio_jk)
        plt_x=(tcList[1:tf]-tf//2)*xUnit; plt_y=ratio_mean[1:tf]*yUnit; plt_err=ratio_err[1:tf]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
        
    icol=1
    for tau_plat in tau_plats:
        tcList_dic={tf:np.arange(tau_plat,tf-tau_plat+1) for tf in tfs if tf-tau_plat+1>tau_plat}
        tfList=list(tcList_dic.keys()); tfList.sort()
        Ndata=np.sum([len(tcList_dic[tf]) for tf in tfList])
        def fitfunc(pars):
            return np.zeros(Ndata)+pars[0]
        def estimator(pars):
            return [pars[0]]
        ratio_jk=np.hstack([ratio_jk_dic[tf][:,tcList_dic[tf]] for tf in tfList])
        y_jk=ratio_jk; p0=[ratio_jk[0,Ndata//2]]
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,p0,estimator=estimator)
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        plt_x=tau_plat*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
        ylim=axs[irow,icol].get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),size=chi2Size,ha='center')
            
plt.tight_layout()
plt.savefig('fig/3pt_plateau2.pdf')
plt.close()

/p/project/pines/li47/code/glwc2/example/basicLatticeAnalysis/util.py:46: RuntimeWarning: divide by zero encountered in double_scalars
  chi2R=chi2/Ndof


In [11]:
# copied from 2pt_analysis with modifications

settings={
    'fitmins_1st':range(8,34+1),
    'fitmins_2st':range(1,20+1),
    'fitmins_3st':range(1,7+1),
    'ylim1':[0.6,1.7],
    'ylim2':[0.9,1.04],
    'ylim3':[0.9,3.0],
}

chi2Size=9
xUnit=aLat; yUnit=aInvLat

func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))

def run(corrQ=True):
    fig, axd = plt.subplot_mosaic([['f1','f1','f1'],['f2','f2','f3']],figsize=(20,8))
    (ax1,ax2,ax3)=(axd[key] for key in ['f1','f2','f3'])
    fig.suptitle('Correlated fit to C2pt' if corrQ else 'Uncorrelated fit to C2pt')
    
    ax1.set_xlabel(r'$t$ [fm]')
    ax2.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax3.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax1.set_ylabel(r'$m_N^{\mathrm{eff}}$ [GeV]')
    ax2.set_ylabel(r'$m_N$ [GeV]')
    ax3.set_ylabel(r'$E_1$ [GeV]')
    ax1.set_ylim(settings['ylim1'])
    ax2.set_ylim(settings['ylim2'])
    ax3.set_ylim(settings['ylim3'])
    
    mN_exp=0.938
    ax1.axhline(y=mN_exp,color='black',linestyle = '--', marker='')
    ax2.axhline(y=mN_exp,color='black',linestyle = '--', marker='', label=r'$m_N^{\mathrm{exp}}=$'+'%0.3f'%mN_exp)
    C2pt_jk=yu.jackknife(C2pt_dat)
    C2pt_mean,C2pt_err=yu.jackme(C2pt_jk)
    C2pt_rela=np.abs(C2pt_err/C2pt_mean)
    temp=[(i,rela) for i,rela in enumerate(C2pt_rela) if rela>0.2]
    fitmax=temp[0][0]-1
    
    func=lambda C2pt: np.log(C2pt/np.roll(C2pt,-1,axis=0))
    mEff_jk=yu.jackmap(func,C2pt_jk)
    (mEff_mean,mEff_err)=yu.jackme(mEff_jk)
    tmin=1; tmax=fitmax+1
    plt_x=np.arange(tmin,tmax)*xUnit; plt_y=mEff_mean[tmin:tmax]*yUnit; plt_err=mEff_err[tmin:tmax]*yUnit
    ax1.errorbar(plt_x,plt_y,plt_err,color='black',fmt='s',mfc='white')

    pars0_initial=[0.4,1e-8,0.5,2,0.8,1]
    
    # 1st fits
    color='r'
    fitmins=settings['fitmins_1st']
    pars0=pars0_initial[:2]; Npar=len(pars0)
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            return func_C2pt_1st(tList,*pars)
        def estimator(pars):
            return [pars[0]]
        y_jk=C2pt_jk[:,tList]
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0,estimator=estimator,mask=None if corrQ else 'uncorrelated')
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        pars0=obs_mean[-Npar:]
        fits.append([obs_mean,obs_err,chi2R,Ndof])
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
    obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
    pars0=obs_mean_MA[-Npar:]
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_err=obs_err_MA[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2,label=r'$m_N^{\mathrm{1st}}=$'+yu.un2str(plt_y,plt_err))
    
    # 2st fits
    color='g'
    fitmins=settings['fitmins_2st']
    pars0=np.hstack([pars0,pars0_initial[2:4]]); Npar=len(pars0)
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            return func_C2pt_2st(tList,*pars)
        def estimator(pars):
            return [pars[0],pars[0]+pars[2]]
        y_jk=C2pt_jk[:,tList]
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0,estimator=estimator,mask=None if corrQ else 'uncorrelated')
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        pars0=obs_mean[-Npar:]
        fits.append([obs_mean,obs_err,chi2R,Ndof])
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[1]*yUnit; plt_err=obs_err[1]*yUnit
        ax3.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax3.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax3.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
    obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
    pars0=obs_mean_MA[-Npar:]
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_err=obs_err_MA[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$m_N^{\mathrm{2st}}=$'+yu.un2str(plt_y,plt_err))
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[1]*yUnit; plt_err=obs_err_MA[1]*yUnit
    ax3.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2)
    ind=np.argmax(probs)
    fitmin_C2pt,fitmax_C2pt=fitmins[ind],fitmax
    pars_C2pt=fits[ind][0][-Npar:]
    
    # # 3st fits
    # color='b'
    # fitmins=settings['fitmins_3st']
    # pars0=np.hstack([pars0,pars0_initial[4:6]]); Npar=len(pars0)
    # fits=[]
    # for fitmin in fitmins:
    #     tList=np.arange(fitmin,fitmax)
    #     def fitfunc(pars):
    #         return func_C2pt_3st(tList,*pars)
    #     def estimator(pars):
    #         return [pars[0],pars[0]+pars[2]]
    #     y_jk=C2pt_jk[:,tList]
    #     obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0,estimator=estimator,mask=None if corrQ else 'uncorrelated')
    #     (obs_mean,obs_err)=yu.jackme(obs_jk)
    #     pars0=obs_mean[-Npar:]
    #     fits.append([obs_mean,obs_err,chi2R,Ndof])
        
    #     plt_x=fitmin*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
    #     ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
    #     ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
    #     ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
    #     plt_x=fitmin*xUnit; plt_y=obs_mean[1]*yUnit; plt_err=obs_err[1]*yUnit
    #     ax3.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
    #     ylim=ax3.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
    #     ax3.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
    # obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
    # plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_err=obs_err_MA[0]*yUnit
    # ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$m_N^{\mathrm{3st}}=$'+yu.un2str(plt_y,plt_err))
    # plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[1]*yUnit; plt_err=obs_err_MA[1]*yUnit
    # ax3.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2)
    
    ax2.legend(loc=(0.6,0.5),fontsize=12)
    
    plt.tight_layout()
    # plt.savefig('fig/fit_2pt_cor.pdf' if corrQ else 'fig/fit_2pt_unc.pdf')
    plt.close()
    
    return (fitmin_C2pt,fitmax_C2pt,pars_C2pt)
    
fitmin_C2pt,fitmax_C2pt,pars_C2pt=run(True)
print(fitmin_C2pt,fitmax_C2pt)
print(pars_C2pt)

9 44
[2.72861424e-01 3.73556431e-10 1.91148037e-01 8.24816400e-01]


In [12]:
# 2st fit
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C3pt_2st=lambda tf,tc,E0a,E0b,a00,dE1a,dE1b,ra01,ra10,ra11: a00*np.exp(-E0a*(tf-tc))*np.exp(-E0b*tc)*(1 + ra01*np.exp(-dE1b*tc) + ra10*np.exp(-dE1a*(tf-tc)) + ra11*np.exp(-dE1a*(tf-tc))*np.exp(-dE1b*tc))

(Nrow,Ncol)=(4,2); (Lrow,Lcol)=(4,8)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False, gridspec_kw={'width_ratios': [2, 1]}, sharex='col', sharey='row')

for irow,case in enumerate(['conn','disc','full','discM']):
    
    xUnit=1; yUnit=aMUL*aInvLat*1000; #yUnit=ZP
    
    if case in ['conn']:
        tfs_temp=tfs
        C3pt_jk_dic_temp={tf:C3pt_jk_dic[tf] for tf in tfs_temp}
        tau_plats=np.arange(1,12+1)
        axs[irow,0].set_ylim(np.array([10,40])*yUnit/(aMUL*aInvLat*1000))
    elif case in ['disc']:
        tfs_temp=tfs_disc
        C3pt_jk_dic_temp={tf:C3pt_disc_jk_dic[tf]-(C2pt_jk[:,tf]*vev_jk)[:,None] for tf in tfs_temp}
        tau_plats=np.arange(1,14+1)
        axs[irow,0].set_ylim(np.array([-10,20])*yUnit/(aMUL*aInvLat*1000))
    elif case in ['full']:
        tfs_temp=tfs
        C3pt_jk_dic_temp={tf:C3pt_jk_dic[tf]+C3pt_disc_jk_dic[tf]-(C2pt_jk[:,tf]*vev_jk)[:,None] for tf in tfs_temp}
        tau_plats=np.arange(1,12+1)
        axs[irow,0].set_ylim(np.array([0,60])*yUnit/(aMUL*aInvLat*1000))
    elif case in ['discM']:
        tfs_temp=tfs
        C3pt_jk_dic_temp={tf:C3pt_disc_jk_dic[tf]-(C2pt_jk[:,tf]*vev_jk)[:,None] for tf in tfs_temp}
        tau_plats=np.arange(1,12+1)
        axs[irow,0].set_ylim(np.array([-10,20])*yUnit/(aMUL*aInvLat*1000))

    chi2Size=12
    for tf in tfs_temp:
        icol=0
        tcList=np.arange(tf+1)
        ratio_jk=[C3pt[tcList]/C2pt[tf] for C2pt,C3pt in zip(C2pt_jk,C3pt_jk_dic_temp[tf])]
        ratio_jk=np.array(ratio_jk)
        ratio_mean,ratio_err=yu.jackme(ratio_jk)
        plt_x=(tcList[1:tf]-tf/2)*xUnit; plt_y=ratio_mean[1:tf]*yUnit; plt_err=ratio_err[1:tf]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
            
    icol=1
    pars0_initial=list(pars_C2pt)+[pars_C2pt[1],1,1]
    pars0=pars0_initial; Npar=len(pars0)
    fits=[]
    for tau_plat in tau_plats:
        print(case,tau_plat,end='                     \r')
        tcList_dic={tf:np.arange(tau_plat,tf-tau_plat+1) for tf in tfs_temp if tf-tau_plat+1>tau_plat}
        tfList=list(tcList_dic.keys()); tfList.sort()
        Ndata=np.sum([len(tcList_dic[tf]) for tf in tfList])
        tfList_2pt=np.arange(fitmin_C2pt,fitmax_C2pt)
        def fitfunc(pars):
            (E0,c0,dE1,rc1,a00,ra01,ra11)=pars
            t2pt=func_C2pt_2st(tfList_2pt,E0,c0,dE1,rc1)
            t3pt=[func_C3pt_2st(tf,tcList_dic[tf],E0,E0,a00,dE1,dE1,ra01,ra01,ra11) for tf in tfList]
            return np.hstack([t2pt]+t3pt)
        def estimator(pars):
            (E0,c0,dE1,rc1,a00,ra01,ra11)=pars
            return [a00/c0,E0,E0+dE1]
        y_jk=np.hstack([C2pt_jk[:,fitmin_C2pt:fitmax_C2pt]]+[C3pt_jk_dic_temp[tf][:,tcList_dic[tf]] for tf in tfList])
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0,estimator=estimator)
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        pars0=obs_mean[-Npar:]
        fits.append([obs_mean,obs_err,chi2R,Ndof])
        plt_x=tau_plat*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white',color='r')
        ylim=axs[irow,icol].get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),size=chi2Size,ha='center')
        
    obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
    pars0=obs_mean_MA[-Npar:]
    plt_x=np.array([-12.5,12.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_err=obs_err_MA[0]*yUnit
    axs[irow,0].fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color='r',alpha=0.2, label=r'$\sigma^{\mathrm{'+case+r'}}_{\pi N}=$'+yu.un2str(plt_y,plt_err))
    plt_x=np.array([tau_plats[0]-0.5,tau_plats[-1]+0.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_err=obs_err_MA[0]*yUnit
    axs[irow,1].fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color='r',alpha=0.2)
    axs[irow,0].legend()
                
plt.tight_layout()
plt.savefig('fig/3pt_2st.pdf')
plt.close()

############

In [13]:
(Nrow,Ncol)=(1,2); (Lrow,Lcol)=(4,8)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False, sharex=True, sharey=True)

irow=0; icol=0
for tf in tfs:
    tcList=np.arange(tf+1)
    ratio_jk=[C3pt[tcList]/C2pt[tf] for C2pt,C3pt in zip(C2pt_jk,C3pt_jk_dic[tf])]
    ratio_mean,ratio_err=yu.jackme(ratio_jk)
    xUnit=1; yUnit=aMUL*aInvLat*1000
    plt_x=(tcList[1:tf]-tf//2)*xUnit; plt_y=ratio_mean[1:tf]*yUnit; plt_err=ratio_err[1:tf]*yUnit
    axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
    
irow=0; icol=1
for tf in tfs:
    tcList=np.arange(tf+1)
    ratio_jk=[C3pt[tcList]/C2pt for C2pt,C3pt in zip(C2pt_jk_dic[tf],C3pt_jk_dic[tf])]
    ratio_mean,ratio_err=yu.jackme(ratio_jk)
    xUnit=1; yUnit=aMUL*aInvLat*1000
    plt_x=(tcList[1:tf]-tf//2)*xUnit; plt_y=ratio_mean[1:tf]*yUnit; plt_err=ratio_err[1:tf]*yUnit
    axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')

plt.tight_layout()
plt.close()

In [14]:
(Nrow,Ncol)=(2,2); (Lrow,Lcol)=(4,8)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False, gridspec_kw={'width_ratios': [2, 1]}, sharex='col', sharey=True)

tau_plat=8
chi2Size=12
xUnit=1; yUnit=aMUL*aInvLat*1000
for irow,case in enumerate(['ratio','coRatio']):
    for tf in tfs:
        icol=0
        tcList=np.arange(tf+1)
        ratio_jk=[C3pt[tcList]/C2pt[tf] for C2pt,C3pt in zip(C2pt_jk,C3pt_jk_dic[tf])] \
            if case=='ratio' else [C3pt[tcList]/C2pt for C2pt,C3pt in zip(C2pt_jk_dic[tf],C3pt_jk_dic[tf])]
        ratio_jk=np.array(ratio_jk)
        ratio_mean,ratio_err=yu.jackme(ratio_jk)
        plt_x=(tcList[1:tf]-tf//2)*xUnit; plt_y=ratio_mean[1:tf]*yUnit; plt_err=ratio_err[1:tf]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
        
        icol=1
        if tf<tau_plat*2+2:
            plt_x=tf*xUnit; plt_y=ratio_mean[tf//2]*yUnit; plt_err=ratio_err[tf//2]*yUnit
            axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
        else:
            tcList=np.arange(tau_plat,tf-tau_plat+1)
            def fitfunc(pars):
                return np.zeros(len(tcList))+pars[0]
            def estimator(pars):
                return [pars[0]]
            y_jk=ratio_jk[:,tcList]; p0=[ratio_jk[0,tf//2]]
            obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,p0,estimator=estimator)
            (obs_mean,obs_err)=yu.jackme(obs_jk)
            plt_x=tf*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
            axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
            ylim=axs[irow,icol].get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
            axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),size=chi2Size,ha='center')
            
plt.tight_layout()
plt.close()

In [15]:
(Nrow,Ncol)=(2,2); (Lrow,Lcol)=(4,8)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False, gridspec_kw={'width_ratios': [2, 1]}, sharex='col', sharey=True)

tau_plats=np.arange(1,13+1)
chi2Size=12
xUnit=1; yUnit=aMUL*aInvLat*1000
for irow,case in enumerate(['ratio','coRatio']):
    ratio_jk_dic={}
    for tf in tfs:
        icol=0
        tcList=np.arange(tf+1)
        ratio_jk=[C3pt[tcList]/C2pt[tf] for C2pt,C3pt in zip(C2pt_jk,C3pt_jk_dic[tf])] \
            if case=='ratio' else [C3pt[tcList]/C2pt for C2pt,C3pt in zip(C2pt_jk_dic[tf],C3pt_jk_dic[tf])]
        ratio_jk=np.array(ratio_jk)
        ratio_jk_dic[tf]=ratio_jk
        ratio_mean,ratio_err=yu.jackme(ratio_jk)
        plt_x=(tcList[1:tf]-tf//2)*xUnit; plt_y=ratio_mean[1:tf]*yUnit; plt_err=ratio_err[1:tf]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
        
    icol=1
    for tau_plat in tau_plats:
        tcList_dic={tf:np.arange(tau_plat,tf-tau_plat+1) for tf in tfs if tf-tau_plat+1>tau_plat}
        tfList=list(tcList_dic.keys()); tfList.sort()
        Ndata=np.sum([len(tcList_dic[tf]) for tf in tfList])
        def fitfunc(pars):
            return np.zeros(Ndata)+pars[0]
        def estimator(pars):
            return [pars[0]]
        ratio_jk=np.hstack([ratio_jk_dic[tf][:,tcList_dic[tf]] for tf in tfList])
        y_jk=ratio_jk; p0=[ratio_jk[0,Ndata//2]]
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,p0,estimator=estimator)
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        plt_x=tau_plat*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_err,fmt='s',mfc='white')
        ylim=axs[irow,icol].get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),size=chi2Size,ha='center')
            
plt.tight_layout()
plt.close()

/p/project/pines/li47/code/glwc2/example/basicLatticeAnalysis/util.py:46: RuntimeWarning: divide by zero encountered in double_scalars
  chi2R=chi2/Ndof


In [16]:
# two-state fit to C2pt (full statistics) and C3pt


# two-state fit to mEff (full statistics) and correlated ratio C3pt/C2pt (matched statistics) [consider correlation only between different times of C3pt/C2pt]